In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import os
import pandas as pd

In [2]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [3]:
seasons = list(range(2021,2022))
seasons

[2021]

In [4]:
# Set up the browser
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

In [5]:
for season in seasons:
    # Visit the website for scraping
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    browser.visit(url)

    # Create a BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    filter_div = soup.find('div', class_='filter')
    links = filter_div.find_all('a')
    urls = [link.get("href") for link in links]

In [6]:
table = soup.find('table', id='schedule')

links = table.find_all("a")
hrefs = [link.get('href') for link in links]
box_scores = [link for link in hrefs if link and "boxscore" in link and ".html" in link]
box_scores = [f"https://www.basketball-reference.com{score}" for score in box_scores]

In [7]:
box_score = box_scores[0]
browser.visit(box_score)

html=browser.html
soup = BeautifulSoup(html, 'html.parser')

In [8]:
score_table = soup.find('table', id='line_score')
tbody = score_table.find('tbody')
rows = tbody.find_all('tr')

line_score = []

for row in rows:
    # Get team name
    team = row.find('th', class_='center').text
    columns = row.find_all('td')
    
    # Create line score dictionary
    if(columns !=[]):
        total = columns[4].text
    
    # Create dictionary for dataframe later
    line_score_dict = { "team": team,
                        "total": total
    }
    
    # Add dictionary to array
    line_score.append(line_score_dict)

In [9]:
# Create Data frame
score_df = pd.DataFrame(line_score)
score_df

,team,total
0,GSW,99
1,BRK,125


In [10]:
# Get basic and advanced box scores
teams = []
for i in range(len(line_score)):
    teams.append(line_score[i]['team'])

In [32]:
teams

['GSW', 'BRK']

In [56]:
team = teams[0]

soup = BeautifulSoup(html, 'html.parser')

basic = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-basic"}, index_col=0)[0]
advanced = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-advanced"}, index_col=0)[0]


In [68]:
advanced.drop([5])

KeyError: '[5] not found in axis'